In [1]:
import pandas as pd
import statsmodels.api as sm

In [2]:
df = pd.read_csv("../Data/Dataset.csv")
df = df.drop("Unnamed: 0", axis=1)
df

,Label (Grouping),Unemployed,Government workers,No health insurance coverage,Families w/ income below poverty,Married-couple household with children of the householder under 18 years,Bachelor's degree or higher,Residence 1 year ago in a different state,Residence 1 year ago abroad,Foreign born,...,Lacking complete plumbing facilities,Housing units with a mortgage,SMOCAPI 35pct. or more for housing units with a mortgage,65 years and over,White non-Hispanics,"$200,000 or more total household income",Mobile home,Homeowner vacancy,SFHA Penetration Rate,Coastal County
0,"Anderson County, Texas",0.017,0.227,0.169,0.106,0.184,0.120,0.008,0.003,0.060,...,0.002,0.430,0.203,0.145,0.573,0.026,0.224,0.014,0.039,0.0
1,"Angelina County, Texas",0.037,0.176,0.192,0.127,0.182,0.180,0.018,0.003,0.083,...,0.002,0.438,0.140,0.162,0.597,0.028,0.189,0.013,0.192,0.0
2,"Aransas County, Texas",0.040,0.180,0.160,0.148,0.069,0.248,0.018,0.007,0.041,...,0.032,0.334,0.331,0.273,0.663,0.063,0.168,0.046,0.571,1.0
3,"Archer County, Texas",0.020,0.155,0.168,0.081,0.199,0.225,0.004,0.002,0.037,...,0.012,0.387,0.147,0.202,0.877,0.072,0.105,0.010,0.413,0.0
4,"Atascosa County, Texas",0.037,0.176,0.195,0.107,0.223,0.148,0.006,0.002,0.068,...,0.004,0.413,0.172,0.147,0.319,0.057,0.333,0.021,0.082,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,"Wood County, Texas",0.022,0.155,0.159,0.100,0.146,0.204,0.020,0.001,0.041,...,0.014,0.403,0.231,0.265,0.814,0.047,0.171,0.006,0.111,0.0
202,"Yoakum County, Texas",0.023,0.175,0.247,0.083,0.337,0.108,0.000,0.000,0.200,...,0.000,0.386,0.041,0.090,0.288,0.103,0.150,0.000,0.164,0.0
203,"Young County, Texas",0.019,0.177,0.184,0.102,0.171,0.257,0.015,0.003,0.077,...,0.014,0.383,0.254,0.204,0.770,0.035,0.080,0.010,0.067,0.0
204,"Zapata County, Texas",0.046,0.228,0.260,0.276,0.179,0.119,0.001,0.004,0.211,...,0.000,0.276,0.173,0.133,0.034,0.032,0.253,0.009,0.031,0.0


In [3]:
# Preprocessing for logit: transform dep. var (SFHA Penetration Rate) into a binary outcome w.r.t. its median
median = df["SFHA Penetration Rate"].median()

for i in range(0, len(df)):
    if df['SFHA Penetration Rate'][i] >= median:
        df['SFHA Penetration Rate'][i] = 1
    else:
        df['SFHA Penetration Rate'][i] = 0

df

/var/folders/fx/qk05lvps2znfsl4f2n_c4dr80000gn/T/ipykernel_31998/595692182.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SFHA Penetration Rate'][i] = 0
/var/folders/fx/qk05lvps2znfsl4f2n_c4dr80000gn/T/ipykernel_31998/595692182.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SFHA Penetration Rate'][i] = 1


,Label (Grouping),Unemployed,Government workers,No health insurance coverage,Families w/ income below poverty,Married-couple household with children of the householder under 18 years,Bachelor's degree or higher,Residence 1 year ago in a different state,Residence 1 year ago abroad,Foreign born,...,Lacking complete plumbing facilities,Housing units with a mortgage,SMOCAPI 35pct. or more for housing units with a mortgage,65 years and over,White non-Hispanics,"$200,000 or more total household income",Mobile home,Homeowner vacancy,SFHA Penetration Rate,Coastal County
0,"Anderson County, Texas",0.017,0.227,0.169,0.106,0.184,0.120,0.008,0.003,0.060,...,0.002,0.430,0.203,0.145,0.573,0.026,0.224,0.014,0.0,0.0
1,"Angelina County, Texas",0.037,0.176,0.192,0.127,0.182,0.180,0.018,0.003,0.083,...,0.002,0.438,0.140,0.162,0.597,0.028,0.189,0.013,1.0,0.0
2,"Aransas County, Texas",0.040,0.180,0.160,0.148,0.069,0.248,0.018,0.007,0.041,...,0.032,0.334,0.331,0.273,0.663,0.063,0.168,0.046,1.0,1.0
3,"Archer County, Texas",0.020,0.155,0.168,0.081,0.199,0.225,0.004,0.002,0.037,...,0.012,0.387,0.147,0.202,0.877,0.072,0.105,0.010,1.0,0.0
4,"Atascosa County, Texas",0.037,0.176,0.195,0.107,0.223,0.148,0.006,0.002,0.068,...,0.004,0.413,0.172,0.147,0.319,0.057,0.333,0.021,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,"Wood County, Texas",0.022,0.155,0.159,0.100,0.146,0.204,0.020,0.001,0.041,...,0.014,0.403,0.231,0.265,0.814,0.047,0.171,0.006,0.0,0.0
202,"Yoakum County, Texas",0.023,0.175,0.247,0.083,0.337,0.108,0.000,0.000,0.200,...,0.000,0.386,0.041,0.090,0.288,0.103,0.150,0.000,1.0,0.0
203,"Young County, Texas",0.019,0.177,0.184,0.102,0.171,0.257,0.015,0.003,0.077,...,0.014,0.383,0.254,0.204,0.770,0.035,0.080,0.010,0.0,0.0
204,"Zapata County, Texas",0.046,0.228,0.260,0.276,0.179,0.119,0.001,0.004,0.211,...,0.000,0.276,0.173,0.133,0.034,0.032,0.253,0.009,0.0,0.0


In [4]:
# Implement model

X = df[["Unemployed",
"Government workers",
"No health insurance coverage",
"Families w/ income below poverty",
"Married-couple household with children of the householder under 18 years",
"Bachelor's degree or higher",
"Residence 1 year ago in a different state",
"Residence 1 year ago abroad",
"Foreign born",
"Vacant housing units",
"Built 1939 or earlier",
"Renteroccupied",
"Lacking complete plumbing facilities",
"Housing units with a mortgage",
"SMOCAPI 35pct. or more for housing units with a mortgage",
"65 years and over",
"White non-Hispanics",
"$200,000 or more total household income",
"Mobile home",
"Homeowner vacancy",
"Coastal County"]]
y = df[["SFHA Penetration Rate"]]
probit = sm.Probit(y, X).fit()
probit.summary()

Optimization terminated successfully.
         Current function value: 0.434554
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                            Probit Regression Results                            
=================================================================================
Dep. Variable:     SFHA Penetration Rate   No. Observations:                  206
Model:                            Probit   Df Residuals:                      185
Method:                              MLE   Df Model:                           20
Date:                   Thu, 23 Mar 2023   Pseudo R-squ.:                  0.3731
Time:                           00:34:18   Log-Likelihood:                -89.518
converged:                          True   LL-Null:                       -142.79
Covariance Type:               nonrobust   LLR p-value:                 8.359e-14
============================================================================================================================================
                                                                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------------------------------
Unemployed                                                                   6.4599      9.665      0.668      0.504     -12.484      25.404
Government workers                                                           1.3244      2.435      0.544      0.587      -3.448       6.097
No health insurance coverage                                                -1.6079      3.852     -0.417      0.676      -9.158       5.942
Families w/ income below poverty                                            -8.8577      3.705     -2.391      0.017     -16.118      -1.597
Married-couple household with children of the householder under 18 years    -4.7558      3.337     -1.425      0.154     -11.296       1.784
Bachelor's degree or higher                                                 -2.1504      2.766     -0.777      0.437      -7.572       3.271
Residence 1 year ago in a different state                                    9.7910     14.072      0.696      0.487     -17.789      37.371
Residence 1 year ago abroad                                                -23.5801     16.210     -1.455      0.146     -55.351       8.191
Foreign born                                                                -0.6256      3.516     -0.178      0.859      -7.517       6.266
Vacant housing units                                                        -5.1952      2.287     -2.272      0.023      -9.678      -0.713
Built 1939 or earlier                                                       -2.1161      3.098     -0.683      0.495      -8.189       3.957
Renteroccupied                                                               3.7289      2.038      1.830      0.067      -0.265       7.722
Lacking complete plumbing facilities                                        36.6994     18.206      2.016      0.044       1.015      72.383
Housing units with a mortgage                                                2.6904      1.827      1.472      0.141      -0.891       6.272
SMOCAPI 35pct. or more for housing units with a mortgage                     1.0843      1.729      0.627      0.531      -2.305       4.473
65 years and over                                                            4.0184      4.591      0.875      0.381      -4.980      13.017
White non-Hispanics                                                         -2.5493      1.017     -2.508      0.012      -4.542      -0.557
$200,000 or more total household income                                     24.1518      7.348      3.287      0.001       9.750      38.553
Mobile home                                                                  1.8058      2.099      0.860      0.390      -2.307       5.919
Homeowner vacancy                                                            5.1949     10.182      0.510      0